In [1]:
# Rede Neural convolucional simples para o problema de reconhecimento de dígitos (MNIST)
import tensorflow as tf
import numpy as np
import random

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# Parâmetros de aprendizagem
taxa_aprendizado = 0.001
quantidade_maxima_epocas = 200
batch_size = 100
keep_prob = tf.placeholder(tf.float32)

In [3]:
# entrada dos place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

In [4]:
# imagem 28x28x1 (preto e branca)
X_img = tf.reshape(X, [-1, 28, 28, 1])

In [5]:
# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([5, 5, 1, 32], stddev=0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
#L1 = tf.nn.dropout(L1, keep_prob)
'''
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
'''

'\nTensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)\nTensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)\nTensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)\n'

In [6]:
# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
#L2 = tf.nn.dropout(L2, keep_prob)

L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])
'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
'''

'\nTensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)\nTensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)\nTensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)\nTensor("Reshape_1:0", shape=(?, 3136), dtype=float32)\n'

In [7]:
# Classificador - Camada Fully Connected entrada 7x7x64 e 1024 neurônios
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 1024],
                     initializer=tf.contrib.layers.xavier_initializer())
L3 = tf.nn.relu(tf.matmul(L2_flat, W3))

L3_drop = tf.nn.dropout(L3, keep_prob)

In [8]:
# Classificador - Camada Fully Connected entrada 1024 -> 10 saídas
W4 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.01))
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L3_drop, W4) + b

In [9]:
# Define a função de custo e o método de otimização
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=taxa_aprendizado).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
# inicializa
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
# treina a rede
print('Rede inicialiada. Treinamento inicializado. Tome um cafe...')
for epoca in range(quantidade_maxima_epocas):
    custo_medio = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.35}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        custo_medio += c / total_batch

    print('Epoca:', '%04d' % (epoca + 1), 'perda =', '{:.9f}'.format(custo_medio))

print('Treinamento finalizado!')

Rede inicialiada. Treinamento inicializado. Tome um cafe...
Epoca: 0001 perda = 0.271997654
Epoca: 0002 perda = 0.075632106
Epoca: 0003 perda = 0.052093111
Epoca: 0004 perda = 0.040722854
Epoca: 0005 perda = 0.032533257
Epoca: 0006 perda = 0.026908229
Epoca: 0007 perda = 0.023123384
Epoca: 0008 perda = 0.021184235
Epoca: 0009 perda = 0.016893014
Epoca: 0010 perda = 0.015970493
Epoca: 0011 perda = 0.013989660
Epoca: 0012 perda = 0.011917619
Epoca: 0013 perda = 0.013658069
Epoca: 0014 perda = 0.010241764
Epoca: 0015 perda = 0.009687454
Epoca: 0016 perda = 0.009196923
Epoca: 0017 perda = 0.008504490
Epoca: 0018 perda = 0.008418002
Epoca: 0019 perda = 0.008625962
Epoca: 0020 perda = 0.006843149
Epoca: 0021 perda = 0.006840873
Epoca: 0022 perda = 0.006613269
Epoca: 0023 perda = 0.008169505
Epoca: 0024 perda = 0.006111656
Epoca: 0025 perda = 0.006610435
Epoca: 0026 perda = 0.007027464
Epoca: 0027 perda = 0.006174156
Epoca: 0028 perda = 0.004954617
Epoca: 0029 perda = 0.005026556
Epoca: 0030 

In [12]:
# Teste o modelo e verifica a taxa de acerto
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Taxa de acerto:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: Mean_1/_19 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_65_Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", lin

In [ ]:
# Teste o modelo e verifica a taxa de acerto
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#Saída da rede
d = tf.cast(tf.argmax(logits, 1),tf.float32)

In [ ]:
#Respostas Corretas
y = tf.cast(tf.argmax(Y,1),tf.float32)

In [ ]:
# Calcula a área da curva ROC (AUC)
auc, update_auc = tf.contrib.metrics.streaming_auc(d, y)

sess.run(tf.local_variables_initializer())

In [ ]:
# Avalia Acurácia
print('Taxa de acerto:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
# Avalia AUC
print('Area da curva ROC:', sess.run(update_auc, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0}))